In [4]:
# Import section
import cv2
import numpy as np
from os import scandir, getcwd, rename
import requests
import time
import json
import re
import pandas as pd

In [5]:
def get_cnn_value_from_json(json_local):
    json.load(json_local)

def stringfy(classes):
    r = ""
    for i in classes:
        r += str(i)
    return r.strip()
def aciertos_from_str(real, est):
    count = 0
    for i in range(len(real)):
        if real[i]==est[i]:
            count += 1
    return count

def distance(str1, str2):
    d=dict()
    for i in range(len(str1)+1):
        d[i]=dict()
        d[i][0]=i
    for i in range(len(str2)+1):
        d[0][i] = i
    for i in range(1, len(str1)+1):
        for j in range(1, len(str2)+1):
            d[i][j] = min(d[i][j-1]+1, d[i-1][j]+1, d[i-1][j-1]+(not str1[i-1] == str2[j-1]))
    return d[len(str1)][len(str2)]

def improve(num, num_pred, scores):
    res = ""
    for i in range(len(str(num))):
        if scores[i] >= 9400:
            res += str(str(num)[i])
        else:
            res += str(str(num_pred)[i])
    return res

def has_letters(word):
    ret = True
    for c in word:
        if not ('0' <= c and c <= '9'):
            return not False
    return not True

def clean_letters(word):
    ret = ""
    for c in word:
        if ('0' <= c and c <= '9'):
            ret += str(c)
    return ret

def clean_real_dni(word):
    word = word.replace("-","")
    word = word.replace("O","0")
    word = word.replace("o","0")
    word = word.replace(" ","")
    return word

np_aciertos_from_str = np.vectorize(aciertos_from_str)
np_distance = np.vectorize(distance)
np_improve = np.vectorize(improve)
np_has_letters = np.vectorize(has_letters)

# REAL

In [6]:
m3100_real = pd.read_csv('D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\process\\muestra900\\levantamiento\\total.csv', dtype={'DNI':str, 'telefono':str})
m3100_real = m3100_real.fillna('')
m3100_real['NombreArchivo'] = m3100_real['nombre_archivo'].apply(lambda x: x[:-4])
m3100_real.head()

,Unnamed: 0,nombre_archivo,DNI,telefono,Observación,email,NombreArchivo
0,0,MarzoCompras_0202072019164448_00043.jpg,42650934,979754672,,Maicusi@hotmail.com,MarzoCompras_0202072019164448_00043
1,1,MarzoCompras_0202072019164454_00245.jpg,20996780,923540008,,,MarzoCompras_0202072019164454_00245
2,2,MarzoCompras_0202072019164458_00359.jpg,42051024,940445230,,jtrujillowong@outlook.com,MarzoCompras_0202072019164458_00359
3,3,MarzoCompras_0202072019164503_00537.jpg,40557762,987136744,,Micasu -80@Hotmail.com,MarzoCompras_0202072019164503_00537
4,4,MarzoCompras_0202072019164503_00546.jpg,08668202,995382267,,CAROLINA CM10_10@HOTMAIL.COM,MarzoCompras_0202072019164503_00546


# ESTIMADO

In [7]:
local = pd.read_csv('azure_result/azure_result_testing.csv', dtype={'DNI':str, 'Telefono':str})
local = local.fillna('')
# local = local[['idCupon','NombreArchivo']]

In [9]:
local = None

In [10]:
bd_azure = pd.read_csv('azure_result/azure_result_testing.csv', dtype={'DNI':str, 'telefono':str})
bd_azure = bd_azure.fillna('')

In [11]:
bd_azure.head(100)

,NombreArchivo,DNI,AcertividadDNI,Telefono,AcertividadTelefono,NombreCompleto,AcertividadNombreCompleto,Direccion,AcertividadDireccion,Distrito,AcertividadDistrito,Correo,AcertividadCorreo,AzureJsonOCR,idCampania,idUsuario,idEstado
0,MarzoCompras-0303072019121007_00008,100672966,75.000000,98233140,0.000000,rath,30.000,Av . Las Wauwtan # 246,71.250000,Chomilloy,60.0,karaujo710hohal,60.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
1,MarzoCompras-0303072019121007_00010,09883016,60.000000,946498591,72.000000,JUAN CARLOS SAlives AnDYA,67.500,AU. PERU N = 36 52,40.000000,SMP,90.0,mauricio2solsbholmail.am,72.857143,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
2,MarzoCompras-0303072019121007_00025,04015880,75.000000,92848937,0.000000,Jaime Roger Compos Benzospi,69.000,M Z H . LT. 12. HE. CAVIN DE HUONTAR.,71.428571,MILA EL SALVADOR.,70.0,,71.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
3,MarzoCompras-0303072019121013_00265,09310742,70.000000,985973705,60.000000,Janet montero flores,66.000,Nelson Quia 415 A Opto 204,71.250000,Surco,60.0,"jalima9-yahoo,es",78.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
4,MarzoCompras-0303072019121014_00331,08281361,70.000000,949352892,72.000000,howyo Morteness,30.000,? Joven nuevo news 14 7 + late I comite y,60.000000,Son Juan ducesamcris,60.0,,71.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
5,MarzoCompras-0303072019121014_00343,0740141,0.000000,536765,72.857143,TERESA HURTADO CASTILLO,69.000,COMA AU . 22 DE AGOSTO) 265,82.500000,Tere hurtado of inthe,60.0,,71.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
6,MarzoCompras-0303072019121017_00450,DN09900151,0.000000,916881398,60.000000,Mariamilla Chauca Palomino,67.500,JR. FELIPE ARIAS 392,70.000000,J. M . P,82.5,"ims-0663@yahoo,es",90.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
7,MarzoCompras-0303072019121017_00459,08982279,67.500000,995327405,78.000000,MAND ANDCELLI GARCIA URBINA,60.000,Ja- L-DAT12 SALCEDO 124 CONDEVILLA,60.000000,SMP,90.0,nerologa@hotmail.com,75.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
8,MarzoCompras-0303072019121018_00497,45277490,67.500000,755211928,60.000000,SHERESNARDA RSA ACUNA YNGA,69.000,Jr. Garala zo de la lega # 414 - Sam Agustin,79.000000,woman,60.0,syay1526@hotmail.com,67.500000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2
9,MarzoCompras-0303072019121018_00501,05333314,60.000000,999171709,60.000000,Marbely Beves Guerra,60.000,Date. 101A Unb- Los Libertadones any . GV LT. OF,66.250000,S. M. P.,80.0,nanditamanee@hotmail.,60.000000,"{""status"": ""Succeeded"", ""recognitionResult"": {...",4,1,2


In [12]:
total_comp_azure = pd.merge(bd_azure, m3100_real, how='left', on='NombreArchivo')
total_comp_azure['dni_lev_azure'] = np_distance(total_comp_azure['DNI_x'],total_comp_azure['DNI_y'])
total_comp_azure['tel_lev_azure'] = np_distance(total_comp_azure['Telefono'],total_comp_azure['telefono'])

total_comp_azure['email'] = np.where(total_comp_azure['email']=="-",
                                    "",
                                    total_comp_azure['email'].apply(str.lower))
total_comp_azure['Correo'] = total_comp_azure['Correo'].apply(str.lower)
total_comp_azure['mail_lev_azure'] = np_distance(total_comp_azure['Correo'],total_comp_azure['email'])

# Cleaning muestra
total_comp_azure['DNI_y'] = total_comp_azure['DNI_y'].apply(str.strip)
total_comp_azure['DNI_y'] = total_comp_azure['DNI_y'].apply(clean_real_dni)

# Columnas con letras
total_comp_azure['letter_in_DNI_real'] = np_has_letters(total_comp_azure['DNI_y'])
total_comp_azure['letter_in_DNI_est'] = np_has_letters(total_comp_azure['DNI_x'])

# Longitud de estimación
total_comp_azure['len_DNI_est'] = total_comp_azure['DNI_x'].apply(len)
total_comp_azure['len_tel_est'] = total_comp_azure['Telefono'].apply(len)
total_comp_azure['len_mail_est'] = total_comp_azure['Correo'].apply(len)

# New rule test
total_comp_azure['DNI_final'] = np.where(total_comp_azure.len_DNI_est>=14,
                                       total_comp_azure['DNI_x'].apply(lambda x : clean_letters(x)[:8]),
                                       total_comp_azure['DNI_x']
                                      )

# Longitud de estimación
total_comp_azure['len_DNI_est_fin'] = total_comp_azure['DNI_final'].apply(len)

total_comp_azure['dni_lev_azure_fin'] = np_distance(total_comp_azure['DNI_final'],total_comp_azure['DNI_y'])

In [13]:
total_comp_azure.columns

Index(['NombreArchivo', 'DNI_x', 'AcertividadDNI', 'Telefono',
       'AcertividadTelefono', 'NombreCompleto', 'AcertividadNombreCompleto',
       'Direccion', 'AcertividadDireccion', 'Distrito', 'AcertividadDistrito',
       'Correo', 'AcertividadCorreo', 'AzureJsonOCR', 'idCampania',
       'idUsuario', 'idEstado', 'Unnamed: 0', 'nombre_archivo', 'DNI_y',
       'telefono', 'Observación', 'email', 'dni_lev_azure', 'tel_lev_azure',
       'mail_lev_azure', 'letter_in_DNI_real', 'letter_in_DNI_est',
       'len_DNI_est', 'len_tel_est', 'len_mail_est', 'DNI_final',
       'len_DNI_est_fin', 'dni_lev_azure_fin'],
      dtype='object')

## Detalle DNI

In [47]:
total_comp_azure[['NombreArchivo','DNI_y','DNI_final','dni_lev_azure_fin']][total_comp_azure.len_DNI_est==1]

,NombreArchivo,DNI_y,DNI_final,dni_lev_azure_fin
78,MarzoCompras-0303072019160832_00750,21286476,_,8
528,MarzoCompras_AV_B5_20190710101445_00875,73804179,1,7
595,MarzoCompras_EL_B2_20190703180748_00688,10528087,_,8
604,MarzoCompras_EL_B2_20190704103516_00490,43937446,!,8
734,MarzoCompras_EL_B2_20190704173329_00684,16620571,_,8
735,MarzoCompras_EL_B2_20190704173331_00732,46176535,_,8
829,MarzoCompras_EL_B3_20190705145337_00713,46741758,_,8
1138,MarzoCompras_LV01072019151525_00192,08638607,_,8
1225,MarzoCompras_LV02072019123159_00037,,:,1
1318,MarzoCompras_LV03072019104005_00141,42385808,_,8


In [8]:
lev_dni_azure = total_comp_azure.groupby(['len_DNI_est']).agg({'NombreArchivo':'count'})
lev_dni_azure['porcentaje'] = np.round(lev_dni_azure['NombreArchivo'] * 100 / lev_dni_azure['NombreArchivo'].sum() , 2)
lev_dni_azure['cuenta'] = lev_dni_azure['NombreArchivo']
lev_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
len_DNI_est,,
0,88,2.78
1,25,0.79
2,2,0.06
3,2,0.06
4,1,0.03
5,1,0.03
6,9,0.28
7,170,5.38
8,2389,75.60


## Detalle Telefono 

In [127]:
total_comp_azure[['NombreArchivo','Telefono','telefono','tel_lev_azure']][total_comp_azure.len_tel_est==0]

,NombreArchivo,Telefono,telefono,tel_lev_azure
122,MarzoCompras_0202072019164511_00837,98083440,980834403,1
153,MarzoCompras_0203072019122719_00317,99803167,99903167,1
166,MarzoCompras_0203072019122734_00881,2764230,2764230,0
182,MarzoCompras_0203072019130749_00573,8975217JJ,997521755,3
191,MarzoCompras_AV_B3_20190705171107_00218,,5311830,7
241,MarzoCompras_AV_B3_20190708093427_00570,,991589375,9
245,MarzoCompras_AV_B3_20190708093431_00718,,989568657,9
264,MarzoCompras_AV_B3_20190708100700_00554,DN:19825169964646696,-,20
283,MarzoCompras_AV_B3_20190708103535_00284,906207081,986207081,1
285,MarzoCompras_AV_B3_20190708103538_00427,,,0


## Detalle E-mail 

In [80]:
total_comp_azure[['NombreArchivo','Correo','email','mail_lev_azure','AcertividadCorreo']][total_comp_azure.len_mail_est==0][total_comp_azure.email==""]

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,NombreArchivo,Correo,email,mail_lev_azure,AcertividadCorreo
2,MarzoCompras-0303072019121007_00025,,,0,71.0
4,MarzoCompras-0303072019121014_00331,,,0,71.0
5,MarzoCompras-0303072019121014_00343,,,0,71.0
12,MarzoCompras-0303072019121019_00541,,,0,71.0
15,MarzoCompras-0303072019121023_00648,,,0,71.0
16,MarzoCompras-0303072019121023_00651,,,0,71.0
17,MarzoCompras-0303072019121023_00652,,,0,71.0
20,MarzoCompras-0303072019121030_00931,,,0,71.0
24,MarzoCompras-0303072019125012_00121,,,0,71.0
29,MarzoCompras-0303072019125015_00264,,,0,71.0


In [74]:
1184 * 100 / 1268

93.37539432176656

## Medicion DNI

In [14]:
lev_dni_azure = total_comp_azure.groupby(['dni_lev_azure_fin']).agg({'NombreArchivo':'count'})
lev_dni_azure['porcentaje'] = np.round(lev_dni_azure['NombreArchivo'] * 100 / lev_dni_azure['NombreArchivo'].sum() , 2)
lev_dni_azure['cuenta'] = lev_dni_azure['NombreArchivo']
lev_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
dni_lev_azure_fin,,
0,1841,58.26
1,746,23.61
2,282,8.92
3,113,3.58
4,41,1.30
5,14,0.44
6,10,0.32
7,16,0.51
8,92,2.91


## Medición Telefono

In [81]:
tel_dni_azure = total_comp_azure.groupby(['tel_lev_azure']).agg({'NombreArchivo':'count'})
tel_dni_azure['porcentaje'] = np.round(tel_dni_azure['NombreArchivo'] * 100 / tel_dni_azure['NombreArchivo'].sum() , 2)
tel_dni_azure['cuenta'] = tel_dni_azure['NombreArchivo']
tel_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
tel_lev_azure,,
0,1699,53.77
1,851,26.93
2,314,9.94
3,125,3.96
4,45,1.42
5,10,0.32
6,6,0.19
7,19,0.60
8,11,0.35


## Medición E-mail

In [83]:
mail_dni_azure = total_comp_azure.groupby(['mail_lev_azure']).agg({'NombreArchivo':'count'})
mail_dni_azure['porcentaje'] = np.round(mail_dni_azure['NombreArchivo'] * 100 / mail_dni_azure['NombreArchivo'].sum() , 2)
mail_dni_azure['cuenta'] = mail_dni_azure['NombreArchivo']
mail_dni_azure[['cuenta','porcentaje']]

,cuenta,porcentaje
mail_lev_azure,,
0,1332,42.15
1,228,7.22
2,238,7.53
3,214,6.77
4,202,6.39
5,186,5.89
6,152,4.81
7,122,3.86
8,93,2.94


## Parametro DNI

In [15]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','DNI_y','DNI_final','AcertividadDNI']]
    # param_dni = param_dni[param_dni.dni_yolo.apply(len)==8]
    param_dni['trues_est'] = np.where(param_dni['AcertividadDNI']>=i, 1, 0)
    param_dni['trues_real'] = np.where(param_dni['DNI_final']==param_dni['DNI_y'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))


40 se tiene 0.89 0.58
41 se tiene 0.89 0.58
42 se tiene 0.89 0.58
43 se tiene 0.89 0.58
44 se tiene 0.89 0.58
45 se tiene 0.89 0.58
46 se tiene 0.89 0.58
47 se tiene 0.89 0.58
48 se tiene 0.89 0.58
49 se tiene 0.89 0.58
50 se tiene 0.89 0.58
51 se tiene 0.89 0.58
52 se tiene 0.89 0.58
53 se tiene 0.89 0.58
54 se tiene 0.89 0.58
55 se tiene 0.89 0.58
56 se tiene 0.89 0.58
57 se tiene 0.89 0.58
58 se tiene 0.89 0.58
59 se tiene 0.89 0.58
60 se tiene 0.89 0.58
61 se tiene 0.59 0.58
62 se tiene 0.59 0.58
63 se tiene 0.59 0.58
64 se tiene 0.59 0.58
65 se tiene 0.59 0.58
66 se tiene 0.58 0.58
67 se tiene 0.57 0.58
68 se tiene 0.54 0.58
69 se tiene 0.53 0.58
70 se tiene 0.53 0.58
71 se tiene 0.46 0.58
72 se tiene 0.45 0.58
73 se tiene 0.43 0.58
74 se tiene 0.42 0.58
75 se tiene 0.42 0.58
76 se tiene 0.27 0.58
77 se tiene 0.27 0.58
78 se tiene 0.26 0.58
79 se tiene 0.24 0.58
80 se tiene 0.24 0.58
81 se tiene 0.18 0.58
82 se tiene 0.17 0.58
83 se tiene 0.15 0.58
84 se tiene 0.15 0.58
85 se tien

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [30]:
param_dni = total_comp_azure[['NombreArchivo','DNI_y','DNI_final','AcertividadDNI']]

param_dni['trues_est'] = np.where(param_dni['AcertividadDNI']>=79, 1, 0)
param_dni['trues_real'] = np.where(param_dni['DNI_final']==param_dni['DNI_y'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


trues_real,0,1
trues_est,,
0,1194,1198
1,125,643


In [31]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100, 2)

trues_real,0,1
trues_est,,
0,37.78,37.91
1,3.96,20.35


## Parámetro Telefono

In [108]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','Telefono','telefono','AcertividadTelefono','tel_lev_azure']]
    param_dni['trues_est'] = np.where(param_dni['AcertividadTelefono']>=i, 1, 0)
    param_dni['trues_real'] = np.where(param_dni['Telefono']==param_dni['telefono'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

40 se tiene 0.81 0.54
41 se tiene 0.81 0.54
42 se tiene 0.81 0.54
43 se tiene 0.81 0.54
44 se tiene 0.81 0.54
45 se tiene 0.81 0.54
46 se tiene 0.81 0.54
47 se tiene 0.81 0.54
48 se tiene 0.81 0.54
49 se tiene 0.81 0.54
50 se tiene 0.81 0.54
51 se tiene 0.81 0.54
52 se tiene 0.81 0.54
53 se tiene 0.81 0.54
54 se tiene 0.81 0.54
55 se tiene 0.81 0.54
56 se tiene 0.81 0.54
57 se tiene 0.81 0.54
58 se tiene 0.81 0.54
59 se tiene 0.81 0.54
60 se tiene 0.81 0.54
61 se tiene 0.54 0.54
62 se tiene 0.54 0.54
63 se tiene 0.54 0.54
64 se tiene 0.54 0.54
65 se tiene 0.54 0.54
66 se tiene 0.53 0.54
67 se tiene 0.52 0.54
68 se tiene 0.5 0.54
69 se tiene 0.5 0.54
70 se tiene 0.5 0.54
71 se tiene 0.44 0.54
72 se tiene 0.43 0.54
73 se tiene 0.41 0.54
74 se tiene 0.4 0.54
75 se tiene 0.4 0.54
76 se tiene 0.27 0.54
77 se tiene 0.27 0.54
78 se tiene 0.26 0.54
79 se tiene 0.24 0.54
80 se tiene 0.24 0.54
81 se tiene 0.18 0.54
82 se tiene 0.17 0.54
83 se tiene 0.14 0.54
84 se tiene 0.14 0.54
85 se tiene 0.1

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [115]:
param_dni['trues_est'] = np.where(param_dni['AcertividadTelefono']>=77, 1, 0)
param_dni['trues_real'] = np.where(param_dni['Telefono']==param_dni['telefono'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


trues_real,0,1
trues_est,,
0,1310,1012
1,151,687


In [116]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100,2)

trues_real,0,1
trues_est,,
0,41.46,32.03
1,4.78,21.74


## Parámetro E-mail

In [117]:
for i in range(40, 92, 1):
    param_dni = total_comp_azure[['NombreArchivo','Correo','email','AcertividadCorreo','mail_lev_azure']]
    param_dni['trues_est'] = np.where(param_dni['AcertividadCorreo']>=i, 1, 0)
    param_dni['trues_real'] = np.where(param_dni['Correo']==param_dni['email'], 1, 0)
    
    te, tr, tt = param_dni['trues_est'].sum(), param_dni['trues_real'].sum(), param_dni['trues_real'].values.size
    # print('{} se tiene {} {} {}'.format(i, te, tr, tt))
    print('{} se tiene {} {}'.format(i, round(te/(tt),2), round(tr/(tt),2)))

40 se tiene 1.0 0.42
41 se tiene 1.0 0.42
42 se tiene 1.0 0.42
43 se tiene 1.0 0.42
44 se tiene 1.0 0.42
45 se tiene 1.0 0.42
46 se tiene 1.0 0.42
47 se tiene 1.0 0.42
48 se tiene 1.0 0.42
49 se tiene 1.0 0.42
50 se tiene 1.0 0.42
51 se tiene 1.0 0.42
52 se tiene 1.0 0.42
53 se tiene 1.0 0.42
54 se tiene 1.0 0.42
55 se tiene 1.0 0.42
56 se tiene 1.0 0.42
57 se tiene 1.0 0.42
58 se tiene 1.0 0.42
59 se tiene 1.0 0.42
60 se tiene 1.0 0.42
61 se tiene 0.79 0.42
62 se tiene 0.79 0.42
63 se tiene 0.79 0.42
64 se tiene 0.79 0.42
65 se tiene 0.79 0.42
66 se tiene 0.77 0.42
67 se tiene 0.72 0.42
68 se tiene 0.66 0.42
69 se tiene 0.65 0.42
70 se tiene 0.65 0.42
71 se tiene 0.58 0.42
72 se tiene 0.18 0.42
73 se tiene 0.13 0.42
74 se tiene 0.13 0.42
75 se tiene 0.13 0.42
76 se tiene 0.08 0.42
77 se tiene 0.08 0.42
78 se tiene 0.08 0.42
79 se tiene 0.06 0.42
80 se tiene 0.06 0.42
81 se tiene 0.04 0.42
82 se tiene 0.04 0.42
83 se tiene 0.03 0.42
84 se tiene 0.03 0.42
85 se tiene 0.03 0.42
86 se tie

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [129]:
param_dni['trues_est'] = np.where(param_dni['AcertividadCorreo']>=70, 1, 0)
param_dni['trues_real'] = np.where(param_dni['Correo']==param_dni['email'], 1, 0)
pd.crosstab(param_dni['trues_est'],param_dni['trues_real'])

C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\GYS\AppData\Local\conda\conda\envs\tfkeras\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


trues_real,0,1
trues_est,,
0,1061,47
1,767,1285


In [130]:
norm = pd.crosstab(param_dni['trues_est'],param_dni['trues_real'],normalize=True)
np.round(norm * 100,2)

trues_real,0,1
trues_est,,
0,33.58,1.49
1,24.27,40.66
